In [1]:
import pandas as pd
import numpy as np

In [2]:
import xgboost 
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

/home/zyc/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
df = pd.read_csv('../csv/Key_Frame2_first_train.csv')

In [4]:
df_X_raw = df.drop('C1', axis=1)

y = df['C1']
X_raw = df_X_raw.astype(float).values

In [6]:
basemodel = xgboost.XGBClassifier()
basemodel.fit(X_raw, y)
fs = [(i, v) for i, v in enumerate(basemodel.feature_importances_)]
fs.sort(key=lambda o: o[1], reverse=True)
fs = fs[0:100]

In [7]:
df_final = pd.read_csv('../csv/Key_Frame2_first_test.csv')
df_X_final_raw = df_final.drop('C1', axis=1)
X_final = df_X_final_raw[df_X_final_raw.columns[[o[0] for o in fs]]].astype(float).values
y_final = df_final['C1']

In [8]:
X = df_X_raw[df_X_raw.columns[[o[0] for o in fs]]].astype(float).values

In [77]:
import time
k = 50
kf = KFold(k, random_state=time.time())

In [78]:
def auc(y, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y, y_pred)
    return metrics.auc(fpr, tpr)

In [79]:
train_auc, test_auc = 0, 0
mat = np.zeros((X.shape[0], 2))
mat_test = np.zeros((X_final.shape[0], 2))
for train_indices, test_indices in kf.split(X):
    X_train, X_test, y_train, y_test = X[train_indices], X[test_indices], y[train_indices], y[test_indices]
    
    ms = [xgboost.XGBClassifier(**{'max_depth': 4}), 
          LogisticRegression(penalty='l2', dual=False),
          #LogisticRegression(penalty='l2', dual=True),
          #LogisticRegression(penalty='l1'),
          #LGBMClassifier(), 
          #SVC(probability=True)
         ]
    for i, m in enumerate(ms):
        m.fit(X_train, y_train)
        for j, yy in enumerate(m.predict_proba(X_test)[:, 1]):
            mat[test_indices[j]][i] = yy 
        
        for j, yy in enumerate(m.predict_proba(X_final)[:, 1]):
            mat_test[j][i] += yy 
mat_test /= k

In [80]:
train_auc, test_auc = 0, 0
for train_indices, test_indices in kf.split(mat):
    X_train, X_test, y_train, y_test = mat[train_indices], mat[test_indices], y[train_indices], y[test_indices]
    m = LogisticRegression()
    m.fit(X_train, y_train)
    train_auc += auc(y_train, m.predict_proba(X_train)[:, 1])
    test_auc += auc(y_test, m.predict_proba(X_test)[:, 1])
train_auc / k, test_auc / k

(0.81902248418362633, 0.81042027543606521)

In [81]:
s = LogisticRegression()
s.fit(mat, y)
print auc(y, s.predict_proba(mat)[:, 1])
print auc(y_final, s.predict_proba(mat_test)[:, 1])

0.819010189307
0.79797979798


In [12]:
mat = np.zeros((X_final.shape[0], 4))
for i, m in enumerate(ms):
    y_final_pred = m.predict_proba(X_final)[:, 1]
    for j, yy in enumerate(y_final_pred):
        mat[j][i] = yy

In [13]:
auc(y_final, stacking.predict_proba(mat)[:, 1])

0.77962962962962967

In [233]:
m = xgboost.XGBClassifier()

In [234]:
m.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [236]:
auc(y_final, m.predict_proba(df_X_final_raw.astype(float).values)[:, 1])

0.59629629629629632